<a href="https://colab.research.google.com/github/palaceIA/CICS_Project/blob/main/src/notebooks/robert/robert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-Tuning RoBERTa Base Model
Esse jupyter faz parte na nossa pesquisa para o Congresso Internacional (CICS) , aqui você ira encontrar nosso experimento referente ao modelo RoBERTa Base e os nossos devidos resultados. O dataset utlizado foi o "dair-ai/emotion" disponivel no HuggingFace

## RoBERTa-base
RoBERTa é um modelo de transformadores pré-treinado em um grande corpus de dados em inglês de forma autossupervisionada. Isso significa que ele foi pré-treinado apenas nos textos brutos, sem humanos rotulando-os de forma alguma (é por isso que ele pode usar muitos dados disponíveis publicamente) com um processo automático para gerar entradas e rótulos desses textos.

Mais precisamente, ele foi pré-treinado com o objetivo de modelagem de linguagem mascarada (MLM). Pegando uma frase, o modelo mascara aleatoriamente 15% das palavras na entrada e então executa a frase mascarada inteira através do modelo e tem que prever as palavras mascaradas. Isso é diferente das redes neurais recorrentes (RNNs) tradicionais que geralmente veem as palavras uma após a outra, ou de modelos autorregressivos como GPT que mascaram internamente os tokens futuros. Ele permite que o modelo aprenda uma representação bidirecional da frase.

Dessa forma, o modelo aprende uma representação interna da língua inglesa que pode então ser usada para extrair características úteis para tarefas posteriores: se você tiver um conjunto de dados de frases rotuladas, por exemplo, poderá treinar um classificador padrão usando as características produzidas pelo modelo BERT como entradas.

Note que este modelo tem como objetivo principal ser ajustado em tarefas que usam a frase inteira (potencialmente mascarada) para tomar decisões, como classificação de sequência, classificação de token ou resposta a perguntas. Para tarefas como geração de texto, você deve olhar para um modelo como GPT2.

## Treinamento do modelo

O modelo RoBERTa foi pré-treinado na reunião de cinco conjuntos de dados:

BookCorpus , um conjunto de dados composto por 11.038 livros não publicados;
Wikipédia em inglês (excluindo listas, tabelas e cabeçalhos);
CC-News , um conjunto de dados contendo 63 milhões de artigos de notícias em inglês rastreados entre setembro de 2016 e fevereiro de 2019.
OpenWebText , uma recriação de código aberto do conjunto de dados WebText usado para treinar GPT-2,
Histórias: um conjunto de dados contendo um subconjunto de dados do CommonCrawl filtrados para corresponder ao estilo de história dos esquemas do Winograd.
Juntos, esses conjuntos de dados pesam 160 GB de texto.

In [ ]:
# Instalando os modulos
!pip install torch transformers datasets matplotlib numpy scikit-learn pandas

### Carregando os modulos

In [ ]:
from datasets import load_dataset
from transformers import (
    AutoModelForSequenceClassification,
    TrainingArguments,
    AutoTokenizer,
    AutoModel
)
from sklearn.metrics import (
    accuracy_score ,
    f1_score ,
    classification_report
)
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch

### Carregando o dataset

In [ ]:
id_data = "dair-ai/emotion"
dataset = load_dataset(id_data)

### Entendendo os dados

In [ ]:
# Estrutura do dataset
print(dataset)

In [ ]:
# Total de classes do dataset
classes = dataset['train'].features['label'].names
classes

In [ ]:
# Alterando o formato do dataset para um tipo pandas
dataset.set_format(type='pandas')
df_pandas = dataset['train'][:]
df_pandas.head()

In [ ]:
# Criando uma tabela para cada classe correspondente
df_pandas['label_name'] = df_pandas['label'].apply(lambda x : classes[x])
df_pandas.head()

In [ ]:
# Verificando o balanceamento das classes
total_classes = df_pandas['label_name'].value_counts()
total_classes

In [ ]:
# Resetando o formato original dos dados
dataset.reset_format()

### Carregando o tokenizador do modelo

In [ ]:
id_model = 'FacebookAI/roberta-base'
tokenizador = AutoTokenizer.from_pretrained(id_model)

In [ ]:
# passaremos a quantidade de batchs dos dados
# Para aplicar essas a tokenizaço de todos os dados
# basta usar o metodo map()
# Função para tokenizar o dataset
def tokenizador_lote(batch):
    temp = tokenizador(
        batch['text'],  # Aqui, 'batch' deve ser um dicionário com uma chave 'text'
        padding=True,
        truncation=True,
    )
    return temp

### Tokenizando os dados

In [ ]:
dataset_tokenizado = dataset.map(
    tokenizador_lote,
    batched = True ,
    batch_size=None
)

### Carregando os modelo

In [ ]:
model= AutoModel.from_pretrained(id_model)

In [ ]:
model

### Configurações iniciais para o ajuste fino

In [ ]:
# Armazenando numero de classes
numero_classes = len(classes)
# Inicializando plataforma CUDA
device = torch.device(
    "cuda" if torch.cuda.is_available() else "cpu"
)
model = AutoModelForSequenceClassification.from_pretrained(
    id_model , num_labels = numero_classes
)

In [ ]:
device

### Configurações do treinamento

In [ ]:
# Tamanho do lote
batch_size = 15
model_name = 'roberta-base-emotions'

training_args = TrainingArguments(
    output_dir=model_name ,
    num_train_epochs=4 ,
    learning_rate=1e-5,
    per_device_train_batch_size= batch_size ,
    per_device_eval_batch_size=batch_size ,
    weight_decay=0.01,
    eval_strategy='epoch' ,
    disable_tqdm=False
)

### Computação de métricas

In [ ]:
def computer_metrics(pred) :
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels,preds,average='weighted')
    acc = accuracy_score(labels,preds)
    return {"acurracy : " : acc , "f1" : f1}


### Treinamento

In [ ]:
trainer = Trainer(
    model = model ,
    args = training_args ,
    compute_metrics = computer_metrics ,
    train_dataset = dataset_tokenizado['train'] ,
    eval_dataset= dataset_tokenizado['validation'] ,
    tokenizer = tokenizer
)

In [ ]:
trainer.train()

### Avaliando modelo

In [ ]:
y_preds = np.argmax(
    pred_ouptus.predictions,axis=1
)
y_true = dataset_tokenizer['test'][:]['label']

In [ ]:
print(classification_report(y_true,y_preds,output_dict=True))

In [ ]:
report_df = pd.DataFrame(report_dict).transpose()
report_df = report_df.round(4)

# Exibe a tabela
print(report_df)

In [ ]:
# Binariza os labels para ROC multiclasse
y_test_bin = label_binarize(y_true, classes=list(range(len(classes))))

# Calcula curva ROC e AUC para cada classe
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(len(classes)):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_pred_proba[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plota as curvas ROC
plt.figure(figsize=(10, 8))
for i in range(len(classes)):
    plt.plot(fpr[i], tpr[i], lw=2,
             label=f"{classes[i]} (AUC = {roc_auc[i]:.2f})")

plt.plot([0, 1], [0, 1], "k--", lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("Taxa de Falsos Positivos (FPR)")
plt.ylabel("Taxa de Verdadeiros Positivos (TPR)")
plt.title("Curva ROC por Classe")
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

In [ ]:
data = []
for i, cls in enumerate(classes):
    # Protege contra índice fora do range se houver poucos thresholds
    fpr_value = fpr[i][1] if len(fpr[i]) > 1 else fpr[i][0]
    tpr_value = tpr[i][1] if len(tpr[i]) > 1 else tpr[i][0]

    data.append({
        "Classe": cls,
        "AUC": round(roc_auc[i], 4),
        "FPR (Exemplo)": round(fpr_value, 4),
        "TPR (Exemplo)": round(tpr_value, 4)
    })

roc_df = pd.DataFrame(data)
print(roc_df)